In [32]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.
En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

1.- El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

2.- El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos

In [2]:
attack=pd.read_csv('../ETL/datos/attacks_limpieza_completa.csv', index_col=0)
df=pd.read_csv('../ETL/datos/df_API_paises.csv', index_col=0)

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea


In [3]:
df_attack=attack[(attack["country"]=="usa") | (attack["country"]=="australia")|\
          (attack["country"]=="south africa") | (attack["country"]=="papua new guinea")|\
          (attack["country"]=="new zealand")]

In [4]:
df_attack.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [41]:
df.sample(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
160,99,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 16}, {'layer': '900m...","[{'layer': '950mb', 'direction': 35, 'speed': ...",17,2,15,1015,rain,7,0,30,4,-28.816624,24.991639,South Africa
185,174,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 305, 'speed':...",17,6,12,1018,none,7,0,310,2,-28.816624,24.991639,South Africa


- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

In [37]:
#el groupby es por pais

In [17]:
assd=pd.Series(df["rh_profile"])
assd


0      [{'layer': '950mb', 'rh': 12}, {'layer': '900m...
1      [{'layer': '950mb', 'rh': 8}, {'layer': '900mb...
2      [{'layer': '950mb', 'rh': 6}, {'layer': '900mb...
3      [{'layer': '950mb', 'rh': 4}, {'layer': '900mb...
4      [{'layer': '950mb', 'rh': 2}, {'layer': '900mb...
                             ...                        
315    [{'layer': '950mb', 'rh': 15}, {'layer': '900m...
316    [{'layer': '950mb', 'rh': 14}, {'layer': '900m...
317    [{'layer': '950mb', 'rh': 13}, {'layer': '900m...
318    [{'layer': '950mb', 'rh': 12}, {'layer': '900m...
319    [{'layer': '950mb', 'rh': 12}, {'layer': '900m...
Name: rh_profile, Length: 320, dtype: object

- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [38]:
x = df['rh_profile'].apply(pd.Series)
x[0]=x[0].apply(ast.literal_eval)
x = df['rh_profile'].apply(pd.Series)

x = df['rh_profile'].apply(pd.Series)
x[0].apply(pd.Series)['rh'][0] 

x = df['rh_profile'].apply(pd.Series)


for values in range(len(x.columns)):
    nombre='rh' + str(x[values].apply(pd.Series)["layer"][0])
    valores = list(x[values].apply(pd.Series)['rh'])
    df.insert(values, nombre, valores) 

KeyError: 'rh'

In [39]:
x = df['rh_profile'].apply(pd.Series)
x[0]=x[0].apply(ast.literal_eval)
x = df['rh_profile'].apply(pd.Series)
x

,0
0,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m..."
1,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb..."
2,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb..."
3,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb..."
4,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb..."
...,...
315,"[{'layer': '950mb', 'rh': 15}, {'layer': '900m..."
316,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m..."
317,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m..."
318,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m..."


In [36]:
x[0].apply(pd.Series)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 12}","{'layer': '900mb', 'rh': 10}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': 10}","{'layer': '750mb', 'rh': 10}","{'layer': '700mb', 'rh': 9}","{'layer': '650mb', 'rh': 6}","{'layer': '600mb', 'rh': 5}","{'layer': '550mb', 'rh': 10}","{'layer': '500mb', 'rh': 13}","{'layer': '450mb', 'rh': 15}","{'layer': '400mb', 'rh': 16}","{'layer': '350mb', 'rh': 16}","{'layer': '300mb', 'rh': 16}","{'layer': '250mb', 'rh': 16}","{'layer': '200mb', 'rh': 15}"
1,"{'layer': '950mb', 'rh': 8}","{'layer': '900mb', 'rh': 7}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 9}","{'layer': '750mb', 'rh': 8}","{'layer': '700mb', 'rh': 3}","{'layer': '650mb', 'rh': 1}","{'layer': '600mb', 'rh': 3}","{'layer': '550mb', 'rh': 7}","{'layer': '500mb', 'rh': 13}","{'layer': '450mb', 'rh': 15}","{'layer': '400mb', 'rh': 16}","{'layer': '350mb', 'rh': 16}","{'layer': '300mb', 'rh': 16}","{'layer': '250mb', 'rh': 16}","{'layer': '200mb', 'rh': 15}"
2,"{'layer': '950mb', 'rh': 6}","{'layer': '900mb', 'rh': 4}","{'layer': '850mb', 'rh': 4}","{'layer': '800mb', 'rh': 7}","{'layer': '750mb', 'rh': 5}","{'layer': '700mb', 'rh': 1}","{'layer': '650mb', 'rh': 0}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 11}","{'layer': '500mb', 'rh': 7}","{'layer': '450mb', 'rh': 12}","{'layer': '400mb', 'rh': 16}","{'layer': '350mb', 'rh': 16}","{'layer': '300mb', 'rh': 16}","{'layer': '250mb', 'rh': 16}","{'layer': '200mb', 'rh': 6}"
3,"{'layer': '950mb', 'rh': 4}","{'layer': '900mb', 'rh': 6}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 4}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 4}","{'layer': '650mb', 'rh': 6}","{'layer': '600mb', 'rh': 8}","{'layer': '550mb', 'rh': 10}","{'layer': '500mb', 'rh': 5}","{'layer': '450mb', 'rh': -1}","{'layer': '400mb', 'rh': 10}","{'layer': '350mb', 'rh': 16}","{'layer': '300mb', 'rh': 16}","{'layer': '250mb', 'rh': 16}","{'layer': '200mb', 'rh': 0}"
4,"{'layer': '950mb', 'rh': 2}","{'layer': '900mb', 'rh': 3}","{'layer': '850mb', 'rh': 5}","{'layer': '800mb', 'rh': 8}","{'layer': '750mb', 'rh': 8}","{'layer': '700mb', 'rh': 7}","{'layer': '650mb', 'rh': 3}","{'layer': '600mb', 'rh': 3}","{'layer': '550mb', 'rh': 8}","{'layer': '500mb', 'rh': 4}","{'layer': '450mb', 'rh': 2}","{'layer': '400mb', 'rh': 3}","{'layer': '350mb', 'rh': 2}","{'layer': '300mb', 'rh': 4}","{'layer': '250mb', 'rh': 3}","{'layer': '200mb', 'rh': 0}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,"{'layer': '950mb', 'rh': 15}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 12}","{'layer': '800mb', 'rh': 11}","{'layer': '750mb', 'rh': 2}","{'layer': '700mb', 'rh': -3}","{'layer': '650mb', 'rh': -4}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -3}","{'layer': '400mb', 'rh': -2}","{'layer': '350mb', 'rh': -3}","{'layer': '300mb', 'rh': -2}","{'layer': '250mb', 'rh': -2}","{'layer': '200mb', 'rh': -1}"
316,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': 8}","{'layer': '750mb', 'rh': 0}","{'layer': '700mb', 'rh': -3}","{'layer': '650mb', 'rh': -4}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -3}","{'layer': '400mb', 'rh': -3}","{'layer': '350mb', 'rh': -3}","{'layer': '300mb', 'rh': -2}","{'layer': '250mb', 'rh': 0}","{'layer': '200mb', 'rh': -1}"
317,"{'layer': '950mb', 'rh': 13}","{'layer': '900mb', 'rh': 14}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 4}","{'layer': '750mb', 'rh': -3}","{'layer': '700mb', 'rh': -4}","{'layer': '650mb', 'rh': -4}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': -4}","{'layer': '500mb', 'rh': -4}","{'layer': '450mb', 'rh': -3}","{'layer': '400mb', 'rh': -3}","{'layer': '350mb', 'rh': -3}","{'layer': '300mb', 'rh': -1}","{'layer': '250mb',

In [35]:
x[0][0]

[{'layer': '950mb', 'rh': 12},
 {'layer': '900mb', 'rh': 10},
 {'layer': '850mb', 'rh': 10},
 {'layer': '800mb', 'rh': 10},
 {'layer': '750mb', 'rh': 10},
 {'layer': '700mb', 'rh': 9},
 {'layer': '650mb', 'rh': 6},
 {'layer': '600mb', 'rh': 5},
 {'layer': '550mb', 'rh': 10},
 {'layer': '500mb', 'rh': 13},
 {'layer': '450mb', 'rh': 15},
 {'layer': '400mb', 'rh': 16},
 {'layer': '350mb', 'rh': 16},
 {'layer': '300mb', 'rh': 16},
 {'layer': '250mb', 'rh': 16},
 {'layer': '200mb', 'rh': 15}]

In [46]:
x = df['wind_profile'].apply(pd.Series)
x[0].apply(pd.Series)['layer'][0] 

x = df['wind_profile'].apply(pd.Series)
x[0].apply(pd.Series)['wind'][0] 

x = df['wind_profile'].apply(pd.Series)


for values in range(len(x.columns)):
    nombre='wind' + str(x[values].apply(pd.Series)['layer'][0])
    valores = list(x[values].apply(pd.Series)['wind'])
    df.insert(values, nombre, valores) 

KeyError: 'layer'

- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.